In [1]:
import os
import joblib
import csv
import random
import requests
import pandas as pd

In [2]:
f = open("columns.txt", "r")
columns = f.read().split('-')[:-1]
f.close()
print(len(columns))

198


In [3]:
good_models = []

for file in os.listdir("."):
    if file.endswith(".pkl"):
        good_models.append(joblib.load(file))

print(len(good_models))

11


## Get Stock Info

In [4]:
blue_chip_stock_200DayMovingAverage_max = 300
api_keys = [
    "2RFQ9MPXDZ19H8ZB", "5HNZLHHXQ49419OW", "AD4QHR1QWWPX3PAZ",
    "DROMFL8PK13N9PVP", "0Z6NF5RSECY0KSUT", "43GLZYIC6DFBYGIT",
    "KUJBKMRA90WZSS7B"]

In [5]:
def transform_stock(symbol):
    column_names = columns
    column_values = columns
    
    # Get General Data
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={random.choice(api_keys)}'
    r = requests.get(url)
    data = r.json()
    
    # Year
    column_values[0] = 1
 
    csv_file = csv.reader(open('pickable_stocks.csv', 'r'), delimiter=',')
    
    for row in csv_file:
        if symbol == row[7]:
            if row[7] != None:
                column_values[0] = ((row[7]%8)+2023)/8
            break
    
    # Country
    country = data["Country"]
    if country == "USA":
        country = 'United States'
    country = f"('{country}',)"
    
    for i in range(6, len(column_values)):
        if column_values[i] == country:
            column_values[i] = 1
        else:
            column_values[i] = 0
    
    # Economy
    column_values[2] = float(data["200DayMovingAverage"])/blue_chip_stock_200DayMovingAverage_max
    
    # Health
    column_values[3] = float(data['ProfitMargin'])
    
    # Freedom
    column_values[4] = min(float(data['DividendYield']) * 100, 1)
    
    # Corruption
    column_values[5] = min(float(data['MarketCapitalization']) / (2 * (10**12)), 1)
    
    # Get Sentiment Analysis Data
    url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers{symbol}&apikey={random.choice(api_keys)}'
    r = requests.get(url)
    data = r.json()
    
    # Happiness
    happiness = []
    for item in data["feed"]:
        for ticker in item["ticker_sentiment"]:
            if ticker["ticker"] == symbol:
                happiness.append(float(ticker["ticker_sentiment_score"]) * float(ticker["relevance_score"]))

    if len(happiness) == 0:
       column_values[1] = 0.5
    else:
        score = ((sum(happiness)/len(happiness)) + 0.35) / 0.7
        column_values[2] = max(min(score, 1), 0)
    
    return pd.DataFrame([column_values], columns=column_names)

In [6]:
def predict_stock(stock):
    stock_transform = transform_stock(stock)
    predictions = []
    
    for model in good_models:
        predictions.append(model.predict(stock_transform.values))
    
    prediction = sum(predictions)[0] / len(predictions)
    
    return [stock, prediction]

In [7]:
def predict_random_stock():
    csv_file = csv.reader(open('pickable_stocks.csv', 'r'), delimiter=',')
    chosen_row = random.choice(list(csv_file))
    
    return predict_stock(chosen_row[0])

In [8]:
predict_stock("AMZN")

['AMZN', 0.21531858392139316]